In [15]:
! rm -r NC
! mkdir NC

In [ ]:
!apt-get install openslide-tools
!pip install openslide-python


from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

In [5]:
import os
import openslide
import xml.etree.cElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

In [7]:
# don't know what factor is, defaults to 1
def parse_xml(anno_path, factor=1):
    tree = ET.ElementTree(file=anno_path)
    annolist = []
    root = tree.getroot()
    for coords in root.iter('Vertices'):
        for coord in coords:
            x = int(float(coord.attrib.get("X")) / factor)
            y = int(float(coord.attrib.get("Y")) / factor)
            annolist.append((x, y))
    return annolist


roi_path = "./drive/MyDrive/mla/CRC_WSIs/1.xml"
roi_coords = parse_xml(roi_path)

In [12]:
def is_not_white_or_gray(patch, threshold=0.95, gray_threshold=200):
    """
    Check if the patch is not predominantly white or light gray.
    """
    gray = cv.cvtColor(np.array(patch), cv.COLOR_RGB2GRAY)
    _, binary = cv.threshold(gray, gray_threshold, 255, cv.THRESH_BINARY)
    foreground_pixels = cv.countNonZero(binary)
    foreground_ratio = foreground_pixels / (patch.shape[0] * patch.shape[1])
    return foreground_ratio < threshold

In [ ]:
import openslide
import os


# the patch is a square
def in_roi(roi_vertices, patch_vertices, patch_size):
    p_x, p_y = patch_vertices
    br_x, br_y = (patch_vertices[0] + patch_size, patch_vertices[1] + patch_size)

    # Check if each ROI vertex is inside the patch area
    for roi_vertex in roi_vertices:
        if (p_x <= roi_vertex[0] <= br_x and
           p_y <= roi_vertex[1] <= br_y):
           return True

    return False



# Function to extract patches from the WSI
def extract_patches(image_path, roi_coords, patch_size=2048):

    slide = openslide.open_slide(image_path)

    mag_level = 0

    width, height = slide.level_dimensions[mag_level]

    for y in range(0, height - patch_size, patch_size):
        for x in range(0, width - patch_size, patch_size):
            # add downsample if needed
            downsampling_factor = slide.level_downsamples[mag_level]
            patch = slide.read_region((int(x * downsampling_factor), int(y * downsampling_factor)), mag_level, (patch_size, patch_size))

            if(is_not_white_or_gray(np.array(patch))):
              if(in_roi(roi_coords, (x, y), patch_size)):
                patch.save(f"./WC/patch_{x}_{y}.png")
                #test
                return
              #else:
              #  patch.save(f"./NC/patch_{x}_,{y}.png")



image_path = "./drive/MyDrive/mla/CRC_WSIs/1.svs"
extract_patches(image_path, roi_coords)


100720 76803
